In [14]:
import pandas as pd
import pdfplumber as pdf
import re
import time
from selenium import webdriver #navegado de Selenium
from selenium.webdriver.common.by import By #Encontrar elementos en el codigo del navegador
from selenium.webdriver.support.wait import WebDriverWait #Esperas del navegador
from selenium.webdriver.support import expected_conditions as EC #Esperar una condición del navegador
from selenium.common.exceptions import NoSuchElementException, InvalidSessionIdException,TimeoutException #Excepciones en Selenium
import datetime

# Extraer y guardar ruts del PDF

In [2]:
"""ruts = []
with pdf.open('./listado.pdf') as listado_proveedores:
    for i in range(len(listado_proveedores.pages)):
        rut = []
        prueba = listado_proveedores.pages[i].extract_text()
        rut = re.findall(r'(\d{2}.\d{3}.\d{3}-[\d{1}A-Za-z])', prueba)
        print(i,len(rut))
        for integrar in rut:
            ruts.append(integrar)
    listado_proveedores.close()
print(len(ruts))"""

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\santi\AppData\Local\Temp\ipykernel_14096\1854325873.py:1: SyntaxWarning: invalid escape sequence '\d'
  """ruts = []


"ruts = []\nwith pdf.open('./listado.pdf') as listado_proveedores:\n    for i in range(len(listado_proveedores.pages)):\n        rut = []\n        prueba = listado_proveedores.pages[i].extract_text()\n        rut = re.findall(r'(\\d{2}.\\d{3}.\\d{3}-[\\d{1}A-Za-z])', prueba)\n        print(i,len(rut))\n        for integrar in rut:\n            ruts.append(integrar)\n    listado_proveedores.close()\nprint(len(ruts))"

In [3]:
"""f = open('codigos.txt',"w")
for codigo in ruts:
    f.write(codigo+'\n')"""

'f = open(\'codigos.txt\',"w")\nfor codigo in ruts:\n    f.write(codigo+\'\n\')'

# Extraer datos de https://www.genealog.cl/Geneanexus/search

In [4]:
with open('codigos.txt', 'r') as archivo:
    # Lee todas las líneas y las almacena en una lista
    lineas = archivo.readlines()

# Elimina los saltos de línea al final de cada elemento
lineas = [linea.strip() for linea in lineas]

print(len(lineas))

75465


In [5]:
def validar(driver,Lista_Xpath, Lista_Guardar):
    for Xpath in Lista_Xpath:
        try:
            elemen = driver.find_element(by=By.XPATH, value=Xpath)
            Lista_Guardar.append(elemen.text)
        except NoSuchElementException:
            elemen = 'null'
            Lista_Guardar.append(elemen)
    return(Lista_Guardar)
    

In [6]:
list_validar_gen = [
    '//*[@id="results-content"]/tr[2]/td[2]/h3/b', #0 Razon social
    '//*[@id="results-content"]/tr[4]/td[2]', #1 Rubro
    '//*[@id="results-content"]/tr[5]/td[2]', #2 Subrubro
    '//*[@id="results-content"]/tr[6]/td[2]', #3 Actividades economicas
    '//*[@id="results-content"]/tr[7]/td[2]', #4 Region
    '//*[@id="results-content"]/tr[8]/td[2]', #5 Comuna
    '//*[@id="results-content"]/tr[9]/td[2]', #6 Ciudad
    '//*[@id="results-content"]/tr[10]/td[2]', #7 Tipo contribuyente
    '//*[@id="results-content"]/tr[11]/td[2]', #8 Subtipo contribuyente
    '//*[@id="results-content"]/tr[12]/td[2]', #9 Fecha inicio
    '//*[@id="results-content"]/tr[13]/td[2]', #10 Cantidad personas
]

In [10]:
url = 'https://www.genealog.cl/Geneanexus/search'
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
driver.get(url)

In [8]:
data_chile = pd.DataFrame()

In [11]:
indice = 0
for i,codigo in enumerate(lineas[::-1]):
    try:
        datos_nuevos = {}
        indice +=1
        if i == 0:
            inp_codigo = driver.find_element(By.XPATH,'/html/body/div[4]/div[1]/div/div/div[1]/form/div/div[1]/div[2]/input')
            inp_codigo.send_keys(codigo)
            btn_buscar = driver.find_element(By.XPATH,'//*[@id="searchSubmitInitial"]/button')
            btn_buscar.click()
        else:
            inp_codigo = driver.find_element(By.XPATH,'/html/body/div[4]/div[1]/div/div/div[1]/form/div/div[1]/div[2]/input')
            inp_codigo.clear()
            inp_codigo.send_keys(codigo)
            btn_buscar = driver.find_element(By.XPATH,'/html/body/div[4]/div[1]/div/div/div[1]/form/div/div[1]/div[3]/button')
            btn_buscar.click()
        time.sleep(5)
        
        tip_entidad = driver.find_element(By.XPATH,'//*[@id="results-content"]/tr[2]/td[3]').text
        if tip_entidad.strip() != 'Empresas':
            print("Es una persona, imposible de extraer su información de esta manera")
            continue
        list_guardar_gen = []
        print("Es Empresa")
        time.sleep(2)
        empresa = driver.find_element(By.XPATH, '/html/body/div[4]/div[3]/div/div[5]/div[1]/table/tbody/tr[2]/td[1]/a')
        empresa.click()
        time.sleep(5)
        
        driver.switch_to.window(driver.window_handles[1])
        validar(driver, list_validar_gen, list_guardar_gen)
        raz_soc = list_guardar_gen[0]
        rubro = list_guardar_gen[1]
        subrubro = list_guardar_gen[2]
        act_eco = list_guardar_gen[3]
        region = list_guardar_gen[4]
        comuna = list_guardar_gen[5]
        ciudad = list_guardar_gen[6]
        tip_cont = list_guardar_gen[7]
        sub_tip_cont = list_guardar_gen[8]
        fec_ini = list_guardar_gen[9]
        cant_pers = list_guardar_gen[10]
        
        try:
            btn_cont = driver.find_element(By.XPATH, '//*[@id="OwnEvent_showContact"]/td/button')
            btn_cont.click()
        except:
            print("Sin boton contacto")
            pass
        time.sleep(1)
        
        cont_class = []
        cont_class = driver.find_elements(By.CLASS_NAME, 'parseOnAsk')
        print(len(cont_class))
        contacto = 'null'
        correo = 'null'
        telefono = 'null'
        new_web = 'null'
        if len(cont_class) > 0:
            try:
                contacto = cont_class[0].find_element(By.XPATH,'span').text
            except NoSuchElementException:
                contacto = 'null'
            for contenido in cont_class:
                
                texto = contenido.text
                if "@" in texto:
                    emails = re.findall(r'\w+@+\w+.+\w',texto)
                    if len(emails) > 1:
                        for email in emails:
                            correo = correo + '\n'+ email
                    elif len(emails) == 1:
                        correo = emails[0]
                    else:
                        correo = 'null'
                    
                elif "+" in texto:
                    tels = re.findall(r'\d+', texto)
                    if len(tels) > 1:
                        for tel in tels:
                            telefono = telefono + '\n'+ tel
                    elif len(tels) == 1:
                        telefono = tels[0]
                    else:
                        telefono = 'null'
                    
                elif "www." in texto and "www.flaticon.com" not in texto:
                    web = re.findall(r'www.+\w+\w{4}', texto)
                    if len(web) == 0:
                        new_web = 'null'
                    elif len(web) == 1:
                        new_web = web[0].replace("launch","").strip()
                    else:
                        for errase_launche in web:
                            new_text = errase_launche.replace("launch","").strip()
                            new_web = new_web+'\n'+new_text
        
        datos_nuevos = {
            'RUT':codigo,
            'RAZON_SOCIAL':raz_soc,
            'RUBRO':rubro,
            'SUBRUBRO':subrubro,
            'ACTIVIDADES_ECONOMICAS':act_eco,
            'REGION':region,
            'COMUNA':comuna,
            'CIUDAD':ciudad,
            'TIPO_CONTRIBUYENTE':tip_cont,
            'CONTACTO':contacto,
            'CORREO':correo,
            'TELEFONO':telefono,
            'SITIO_WEB':new_web
        }
        print(i,datos_nuevos)
        df_new = pd.DataFrame(datos_nuevos,index=[indice])
        data_chile = pd.concat((data_chile,df_new))
        print(data_chile.shape)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    except Exception as ex:
        print("ERROR : ", ex)
        reg_error = {'Fecha':datetime.datetime.now(),
                     'Error':ex}
        df_errores = pd.concat()
driver.close()
data_chile.to_csv('./datos_chile.csv',sep=';', encoding='utf_8_seg')

Es Empresa
Sin boton contacto
0
0 {'RUT': '96.961.400-6', 'RAZON_SOCIAL': 'HARTING ADHESIVOS S A', 'RUBRO': 'INDUSTRIAS MANUFACTURERAS NO METALICAS', 'SUBRUBRO': 'FABRICACION DE OTROS PRODUCTOS QUIMICOS', 'ACTIVIDADES_ECONOMICAS': 'FABRICACION DE OTROS PRODUCTOS QUIMICOS N.C.P..', 'REGION': 'XIII REGION METROPOLITANA', 'COMUNA': 'QUILICURA', 'CIUDAD': 'PERSONA JURIDICA COMERCIAL', 'TIPO_CONTRIBUYENTE': 'SOCIEDADES ANONIMAS CERRADAS', 'CONTACTO': '', 'CORREO': '', 'TELEFONO': '', 'SITIO_WEB': ''}
(3, 13)
Es Empresa
4
vbrevis@gasvalpo.cl
['566006007000', '56322277000']
www.gasvalpo.cl
1 {'RUT': '96.960.800-6', 'RAZON_SOCIAL': 'Gasvalpo S.a.', 'RUBRO': 'SUMINISTRO DE ELECTRICIDAD, GAS Y AGUA', 'SUBRUBRO': 'FABRICACION DE GAS, DISTRIBUCION DE COMBUSTIBLES GASEOSOS POR TUBERIAS', 'ACTIVIDADES_ECONOMICAS': 'FABRICACION DE GAS, DISTRIBUCION DE COMBUSTIBLES GASEOSOS POR TUBERIAS.\nGAS NATURAL.', 'REGION': 'V REGION VALPARAISO', 'COMUNA': 'VINA DEL MAR', 'CIUDAD': 'VIñA DEL MAR', 'TIPO_CONTRIBU

KeyboardInterrupt: 

In [13]:
data_chile.head()

,RUT,RAZON_SOCIAL,RUBRO,SUBRUBRO,ACTIVIDADES_ECONOMICAS,REGION,COMUNA,CIUDAD,TIPO_CONTRIBUYENTE,CONTACTO,CORREO,TELEFONO,SITIO_WEB
1,96.961.400-6,HARTING ADHESIVOS S A,INDUSTRIAS MANUFACTURERAS NO METALICAS,FABRICACION DE OTROS PRODUCTOS QUIMICOS,FABRICACION DE OTROS PRODUCTOS QUIMICOS N.C.P..,XIII REGION METROPOLITANA,QUILICURA,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,,,,
2,96.960.800-6,Gasvalpo S.a.,"SUMINISTRO DE ELECTRICIDAD, GAS Y AGUA","FABRICACION DE GAS, DISTRIBUCION DE COMBUSTIBL...","FABRICACION DE GAS, DISTRIBUCION DE COMBUSTIBL...",V REGION VALPARAISO,VINA DEL MAR,VIñA DEL MAR,PERSONA JURIDICA COMERCIAL,Luis Kipreos Almallotis,vbrevis@gasvalpo.cl,\n566006007000\n56322277000,www.gasvalpo.cl
1,96.961.400-6,HARTING ADHESIVOS S A,INDUSTRIAS MANUFACTURERAS NO METALICAS,FABRICACION DE OTROS PRODUCTOS QUIMICOS,FABRICACION DE OTROS PRODUCTOS QUIMICOS N.C.P..,XIII REGION METROPOLITANA,QUILICURA,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,,,,
2,96.960.800-6,Gasvalpo S.a.,"SUMINISTRO DE ELECTRICIDAD, GAS Y AGUA","FABRICACION DE GAS, DISTRIBUCION DE COMBUSTIBL...","FABRICACION DE GAS, DISTRIBUCION DE COMBUSTIBL...",V REGION VALPARAISO,VINA DEL MAR,VIñA DEL MAR,PERSONA JURIDICA COMERCIAL,Luis Kipreos Almallotis,vbrevis@gasvalpo.cl,\n566006007000\n56322277000,www.gasvalpo.cl
3,96.959.870-1,B-aron Conseil Chile S.a.,"ACTIVIDADES INMOBILIARIAS, EMPRESARIALES Y DE ...",ACTIVIDADES JURIDICAS Y DE ASESORAMIENTO EMPRE...,ACTIVIDADES DE ASESORAMIENTO EMPRESARIAL Y EN ...,XIII REGION METROPOLITANA,LAS CONDES,SANTIAGO,PERSONA JURIDICA COMERCIAL,null,,56223257638,www.baronconseil.cl


Pruebas

In [ ]:
contacto = driver.find_element(By.CLASS_NAME, '')

In [27]:
driver.close()

In [ ]:
driver.switch_to.window(driver.window_handles[1])
web_class = driver.find_elements(By.CLASS_NAME, 'parseOnAsk')
print(len(web_class))
a = 0
for i in web_class:
    a += 1
    print(a)
    contacto = web_class[0].find_element(By.XPATH,'span').text
    print('Contacto : ', contacto)
    texto = i.text
    new_web = []
    if "@" in texto:
        email = re.findall(r'\w+@+\w+.+\w',texto)
        print(email)
    if "+" in texto:
        tels = re.findall(r'\d+', texto)
        print(tels)
    if "www." in texto and "www.flaticon.com" not in texto:
        web = re.findall(r'www.+\w+\w{4}', texto)
        for errase_launche in web:
            new_text = errase_launche.replace("launch","").strip()
            new_web.append(new_text)
        print(new_web)


In [ ]:
driver.switch_to.window(driver.window_handles[1])
prueba = []
validar(driver, list_validar_cont, prueba, 1)
print(prueba)